# Формулировка проблемы

Low-level wrapper для Python должен являться **биндингом** имеющегося API в Питон, учитывающим паттерны программирования в Питоне. При этом он не должен являться самостоятельной библиотекой, требующей поддержки и диктующей свои правила, как это происходит с существующий Low-level Python API.

В идеале:
  - человек по [документации C-API](http://docs.bigartm.org/en/latest/ref/c_interface.html) должен без дополнительных знаний брать и пользоваться оберткой в питоне
  - при изменении C-API необходимо мимнимум правок в Python-обертку
  - у пользователя не должно быть попоболи от жутких конструкций, не свойственных Python
  
Предлагаемое решение:
Обертка C-API для питона должна генерироваться автоматчиески по достаточно простому описанию. Обертка выполняется в виде одного небольшого модуля (не более 1к строк, лучше — меньше), содержащего класс `LibArtm`. 

На основе Low-level wrapper будет строиться удобная python-библиотека для тематического моделирования (реализация `ArtmModel`).

In [1]:
ARTM_LIBRARY_PATH = '/home/romovpa/bigartm/build/src/artm/libartm.so'

In [2]:
import ctypes
from artm import messages_pb2

# Сообщения об ошибках

In [3]:
ARTM_SUCCESS = 0
ARTM_STILL_WORKING = -1

class ArtmException(BaseException): pass

class InternalError(ArtmException): pass
class ArgumentOutOfRangeException(ArtmException): pass
class InvalidMasterIdException(ArtmException): pass
class CorruptedMessageException(ArtmException): pass
class InvalidOperationException(ArtmException): pass
class DiskReadException(ArtmException): pass
class DiskWriteException(ArtmException): pass

ARTM_EXCEPTION_BY_CODE = {
    -2: InternalError,
    -3: ArgumentOutOfRangeException,
    -4: InvalidMasterIdException,
    -5: CorruptedMessageException,
    -6: InvalidOperationException,
    -7: DiskReadException,
    -8: DiskWriteException,
}

# Обертка для удобного вызова низкоуровнего API

In [4]:
import ctypes
from google import protobuf
from artm import messages_pb2

class LibArtm(object):
    
    def __init__(self, lib_name):
        self.cdll = ctypes.CDLL(lib_name)
        
    def __getattr__(self, name):
        func = getattr(self.cdll, name)
        if func is None:
            raise AttributeError('%s is not a function of libartm' % name)
        return self._wrap_call(func)
    
    def _check_error(self, error_code):
        if error_code < -1:
            lib.cdll.ArtmGetLastErrorMessage.restype = ctypes.c_char_p
            error_message = lib.cdll.ArtmGetLastErrorMessage()
            
            # remove exception name from error message
            error_message = error_message.split(':', 1)[-1].strip()
            
            exception_class = ARTM_EXCEPTION_BY_CODE.get(error_code)
            if exception_class is not None:
                raise exception_class(error_message)
            else:
                raise RuntimeError(error_message)
    
    def _wrap_call(self, func):
        def artm_api_call(*args):
            cargs = []
            for arg in args:
                if isinstance(arg, basestring):
                    arg_cstr_p = ctypes.create_string_buffer(arg)
                    cargs.append(arg_cstr_p)
                    
                elif isinstance(arg, protobuf.message.Message):
                    message_str = arg.SerializeToString()
                    message_cstr_p = ctypes.create_string_buffer(message_str)
                    cargs += [len(message_str), message_cstr_p]
                    
                else:
                    cargs.append(arg)
            
            result = func(*cargs)
            self._check_error(result)
            return result
        
        return artm_api_call
            
    def _copy_request_result(self, length):
        message_blob = ctypes.create_string_buffer(length)
        error_code = self.lib_.ArtmCopyRequestResult(length, message_blob)
        self._check_error(error_code)
        return message_blob
        

In [5]:
lib = LibArtm(ARTM_LIBRARY_PATH)

In [6]:
config = messages_pb2.MasterComponentConfig()
config.processors_count = -1

master_id = lib.ArtmCreateMasterComponent(config)

In [7]:
lib.ArtmCreateModel(master_id, messages_pb2.ModelConfig())

0

# Спецификация API и автоматическая генерация обертки

Модификация обертки, приведенной выше, решает следующие задачи:

 - Автоматическа проверка аргументов
 - Осмысленное возвращаемое значение: 
   - `ArtmCreateMasterComponent` возвращает `int`
   - функции вроде `ArtmRequestThetaMatrix` возвращают сообщение
   - функции без возвращаемого знвчения возвращают `None` (вместо `ARTM_SUCCESS`)
 - Автодокументирование: при вызове справки, отображается нормальная спецификация функции
 - Быстрое создание сообщений: вместо сообщения можно передать `dict`, который умным образом преобразуется в соответствующее сообщение (гораздо удобнее в питоне)

##### Вводим элементы языка, которыми будет описано API

In [72]:
class CallSpec(object):
    def __init__(self, name, arguments, result=None, request=None):
        self.name = name
        self.arguments = arguments
        self.result_type = result
        self.request_type = request

##### Описываем API при помощи специального dsl

Список `ARTM_API` нужно будет редактировать при изменении low-level API. Заметьте: необходимо будет минимальное число правок, в отличие от существующего `artm.library`.

Некоторые функции из API объявляются _специальными_, для них по понятной причине не делается обертка:
  - `ArtmGetLastErrorMessage`
  - `ArtmCopyRequestResult`

In [77]:
ARTM_API = [
    CallSpec(
        'ArtmCreateMasterComponent', 
        [('config', messages_pb2.MasterComponentConfig)],
        result=ctypes.c_int,
    ),
    CallSpec(
        'ArtmReconfigureMasterComponent',
        [('master_id', int), ('config', messages_pb2.MasterComponentConfig)],
    ),
    CallSpec(
        'ArtmDisposeMasterComponent',
        [('master_id', int)],
    ),
    CallSpec(
        'ArtmCreateModel',
        [('master_id', int), ('config', messages_pb2.ModelConfig)],
    ),
    CallSpec(
        'ArtmReconfigureModel',
        [('master_id', int), ('config', messages_pb2.ModelConfig)],
    ),
    CallSpec(
        'ArtmDisposeModel',
        [('master_id', int), ('name', str)],
    ),
    CallSpec(
        'ArtmCreateRegularizer',
        [('master_id', int), ('config', messages_pb2.RegularizerConfig)],
    ),
    CallSpec(
        'ArtmReconfigureRegularizer',
        [('master_id', int), ('config', messages_pb2.RegularizerConfig)],
    ),
    CallSpec(
        'ArtmDisposeRegularizer',
        [('master_id', int), ('name', str)],
    ),
    CallSpec(
        'ArtmCreateDictionary',
        [('master_id', int), ('config', messages_pb2.DictionaryConfig)],
    ),
    CallSpec(
        'ArtmReconfigureDictionary',
        [('master_id', int), ('config', messages_pb2.DictionaryConfig)],
    ),
    CallSpec(
        'ArtmDisposeDictionary',
        [('master_id', int), ('name', str)],
    ),
    CallSpec(
        'ArtmAddBatch',
        [('master_id', int), ('args', messages_pb2.AddBatchArgs)],
    ),
    CallSpec(
        'ArtmInvokeIteration',
        [('master_id', int), ('args', messages_pb2.InvokeIterationArgs)],
    ),
    CallSpec(
        'ArtmSynchronizeModel',
        [('master_id', int), ('args', messages_pb2.SynchronizeModelArgs)],
    ),
    CallSpec(
        'ArtmInitializeModel',
        [('master_id', int), ('args', messages_pb2.InitializeModelArgs)],
    ),
    CallSpec(
        'ArtmExportModel',
        [('master_id', int), ('args', messages_pb2.ExportModelArgs)],
    ),
    CallSpec(
        'ArtmImportModel',
        [('master_id', int), ('args', messages_pb2.ImportModelArgs)],
    ),
    CallSpec(
        'ArtmWaitIdle',
        [('master_id', int), ('args', messages_pb2.WaitIdleArgs)],
    ),
    CallSpec(
        'ArtmOverwriteTopicModel',
        [('master_id', int), ('model', messages_pb2.TopicModel)],
    ),
    CallSpec(
        'ArtmRequestThetaMatrix',
        [('master_id', int), ('args', messages_pb2.GetThetaMatrixArgs)],
        request=messages_pb2.ThetaMatrix,
    ),
    CallSpec(
        'ArtmRequestTopicModel',
        [('master_id', int), ('args', messages_pb2.GetTopicModelArgs)],
        request=messages_pb2.TopicModel,
    ),
    CallSpec(
        'ArtmRequestRegularizerState',
        [('master_id', int), ('name', str)],
        request=messages_pb2.RegularizerInternalState,
    ),
    CallSpec(
        'ArtmRequestScore',
        [('master_id', int), ('args', messages_pb2.GetScoreValueArgs)],
        request=messages_pb2.ScoreData,
    ),
    CallSpec(
        'ArtmRequestParseCollection',
        [('args', messages_pb2.CollectionParserConfig)],
        request=messages_pb2.DictionaryConfig,
    ),
    CallSpec(
        'ArtmRequestLoadDictionary',
        [('filename', str)],
        request=messages_pb2.DictionaryConfig,
    ),
    CallSpec(
        'ArtmRequestLoadBatch',
        [('filename', str)],
        request=messages_pb2.Batch,
    ),
    CallSpec(
        'ArtmSaveBatch',
        [('filename', str), ('batch', messages_pb2.Batch)],
    ),
    CallSpec(
        'ArtmSaveBatch',
        [('filename', str), ('batch', messages_pb2.Batch)],
    ),
]

In [78]:
spec = CallSpec(
        'ArtmRequestScore',
        [('master_id', int), ('args', messages_pb2.GetScoreValueArgs)],
        messages_pb2.ScoreData,
        request=True,
    )

In [79]:
spec.arguments

[('master_id', int), ('args', artm.messages_pb2.GetScoreValueArgs)]

##### Конвертация dict -> protobuf message

In [80]:
from types import StringTypes
import logging

def dict_to_message(record, message_type):
    """Convert dict to protobuf message"""
    
    def parse_list(values, message):
        if isinstance(values[0], dict):
            for v in values:
                cmd = message.add()
                parse_dict(v,cmd)
        else:
            message.extend(values)

    def parse_dict(values, message):
        for k, v in values.iteritems():
            if isinstance(v, dict):
                parse_dict(v, getattr(message, k))
            elif isinstance(v, list):
                parse_list(v, getattr(message, k))
            else:
                try:
                    setattr(message, k, v)
                except AttributeError:
                    raise TypeError('Cannot convert dict to protobuf message {message_type}: bad field "{field}"'.format(
                            message_type=str(message_type),
                            field=k,
                        ))
    
    message = message_type()
    parse_dict(record, message)
    
    return message

In [81]:
config = messages_pb2.MasterComponentConfig()
config.processor_queue_max_size = 123
config.processors_count = 10
config.SerializeToString()

'0\n8{'

In [82]:
config = dict_to_message({'processor_queue_max_size': 123, 'processors_count': 10}, messages_pb2.MasterComponentConfig)
config.SerializeToString()

'0\n8{'

In [83]:
dict_to_message({}, messages_pb2.MasterComponentConfig)

In [84]:
dict_to_message({'sdfsdfsdf': 24134}, messages_pb2.MasterComponentConfig)

TypeError: Cannot convert dict to protobuf message <class 'artm.messages_pb2.MasterComponentConfig'>: bad field "sdfsdfsdf"

##### Объект класса `LibArtm` при помощи `__getattr__` автоматически создает обертки над вызовами API

In [85]:
import ctypes
from google import protobuf
from artm import messages_pb2

class LibArtm(object):

    def __init__(self, lib_name):
        self.cdll = ctypes.CDLL(lib_name)
        self._spec_by_name = {spec.name: spec for spec in ARTM_API}
        
    def __getattr__(self, name):
        spec = self._spec_by_name.get(name)
        if spec is None:
            raise AttributeError('%s is not a function of libartm' % name)
        func = getattr(self.cdll, name)
        return self._wrap_call(func, spec)
    
    def _check_error(self, error_code):
        if error_code < -1:
            lib.cdll.ArtmGetLastErrorMessage.restype = ctypes.c_char_p
            error_message = lib.cdll.ArtmGetLastErrorMessage()
            
            # remove exception name from error message
            error_message = error_message.split(':', 1)[-1].strip()
            
            exception_class = ARTM_EXCEPTION_BY_CODE.get(error_code)
            if exception_class is not None:
                raise exception_class(error_message)
            else:
                raise RuntimeError(error_message)

    def _copy_request_result(self, length):
        message_blob = ctypes.create_string_buffer(length)
        error_code = self.lib_.ArtmCopyRequestResult(length, message_blob)
        self._check_error(error_code)
        return message_blob

    def _wrap_call(self, func, spec):
        
        def artm_api_call(*args):
            # check the number of arguments
            n_args_given = len(args)
            n_args_takes = len(spec.arguments)
            if n_args_given != n_args_takes:
                raise TypeError('{func_name} takes {n_takes} argument ({n_given} given)'.format(
                        func_name=spec.name,
                        n_takes=n_args_takes,
                        n_given=n_args_given,
                    ))
            
            cargs = []
            for (arg_index, arg), (arg_name, arg_type) in zip(enumerate(args), spec.arguments):
                # try to cast argument to the required type
                arg_casted = arg
                if issubclass(arg_type, protobuf.message.Message) and isinstance(arg, dict):
                    # dict -> protobuf message
                    arg_casted = dict_to_message(arg, arg_type)
                
                # check argument type
                if not isinstance(arg_casted, arg_type):
                    raise TypeError('Argument {arg_index} ({arg_name}) should have type {arg_type} but {given_type} given'.format(
                            arg_index=arg_index,
                            arg_name=arg_name,
                            arg_type=str(arg_type),
                            given_type=str(type(arg)),
                        ))
                arg = arg_casted
                
                # construct c-style arguments                
                if issubclass(arg_type, basestring):
                    arg_cstr_p = ctypes.create_string_buffer(arg)
                    cargs.append(arg_cstr_p)
                    
                elif issubclass(arg_type, protobuf.message.Message):
                    message_str = arg.SerializeToString()
                    message_cstr_p = ctypes.create_string_buffer(message_str)
                    cargs += [len(message_str), message_cstr_p]
                    
                else:
                    cargs.append(arg)
            
            # make api call
            if spec.result_type is not None:
                func.restype = spec.result_type
            result = func(*cargs)
            self._check_error(result)
            
            # return result value
            if spec.request_type is not None:
                return self._copy_request_result(length=result)
            if spec.result_type is not None:
                return result
        
        return artm_api_call
        

### Тестирование

##### Создаем объект `LibArtm`

In [86]:
lib = LibArtm(ARTM_LIBRARY_PATH)

##### Создаем мастер

In [87]:
master_id = lib.ArtmCreateMasterComponent({'cache_theta': True, 'processors_count': 10})
master_id

({'cache_theta': True, 'processors_count': 10},)
[('config', <class 'artm.messages_pb2.MasterComponentConfig'>)]
[((0, {'cache_theta': True, 'processors_count': 10}), ('config', <class 'artm.messages_pb2.MasterComponentConfig'>))]


6

##### Забыл передать аргумент `master_id`

In [88]:
lib.ArtmCreateModel({'topics_count': 20, 'class_id': ['words', 'labels'], 'class_weight': [1, 0.2]})

TypeError: ArtmCreateModel takes 2 argument (1 given)

##### Исправился, создал модель

In [89]:
lib.ArtmCreateModel(master_id, {'topics_count': 20, 'class_id': ['words', 'labels'], 'class_weight': [1, 0.2]})

(6, {'class_id': ['words', 'labels'], 'topics_count': 20, 'class_weight': [1, 0.2]})
[('master_id', <type 'int'>), ('config', <class 'artm.messages_pb2.ModelConfig'>)]
[((0, 6), ('master_id', <type 'int'>)), ((1, {'class_id': ['words', 'labels'], 'topics_count': 20, 'class_weight': [1, 0.2]}), ('config', <class 'artm.messages_pb2.ModelConfig'>))]


In [90]:
lib.ArtmRequestTopicModel(master_id, {})

(6, {})
[('master_id', <type 'int'>), ('args', <class 'artm.messages_pb2.GetTopicModelArgs'>)]
[((0, 6), ('master_id', <type 'int'>)), ((1, {}), ('args', <class 'artm.messages_pb2.GetTopicModelArgs'>))]


InvalidOperationException: Topic model does not exist